# Running an Inference on Custom Data

> **Note:** In this tutorial, we will be using a sample `RandLANet` `SemanticKITTI` weight file which we need to:
>
> 1. Download from: https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth
> 2. Place the downloaded `randlanet_semantickitti_202201071330utc.pth` file into `'Open3D-ML/docs/tutorial/notebook/'` subdirectory, or any other place and change the `ckpt_path` accordingly.
>
> For other model/dataset weight files, please check out https://github.com/isl-org/Open3D-ML#semantic-segmentation-1


An inference predicts the results based on the trained model. **Inference always depends on a prior step - _training_**.

> **Please see the [Training a Semantic Segmentation Model Using PyTorch](train_ss_model_using_pytorch.ipynb) and [Training a Semantic Segmentation Model Using TensorFlow](train_ss_model_using_tensorflow.ipynb) for training tutorials.**

To recap the model training process, - in our data model, we:

1. define a dataset;
2. create a training split part of the data;
3. define a model;
4. create a `SemanticSegmentation` `pipeline` object;
5. execute the `pipeline.run_train()` method which runs the training process.

While training, the model saves the checkpoint files every few epochs, in the *logs* directory. We use these trained weights to restore the model for inference.

Our first step in Inference on a Custom Data implementation is to import *Open3D-ML* and *Numpy* libraries: 


In [3]:
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import numpy as np

We then create a checkpoint path pointing to the data weights file we downloaded (generated at the end of  the Training stage):

In [9]:
# Download weights using link from model zoo (collection of weights for all combinations of model and dataset).
# Link : https://github.com/isl-org/Open3D-ML#semantic-segmentation-1
# Randlanet SemanticKITTI[PyTorch] : https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth
ckpt_path = './randlanet_semantickitti_202201071330utc.pth'

Now, we define a `dataset` object identical to how it was done in our previous *Training a Semantic Segmentation Model* tutorials:

In [10]:
# We define dataset (similar to train_ss_using_pytorch tutorial)
dataset = ml3d.datasets.SemanticKITTI(dataset_path='SemanticKITTI/', cache_dir='./logs/cache',training_split=['00'], validation_split=['01'], test_split=['01'])


Then, initialize `model` and `pipeline` objects:

In [11]:
# Initializing the model and pipeline
model = ml3d.models.RandLANet(in_channels=3)
pipeline = ml3d.pipelines.SemanticSegmentation(model)

Next, we restore the model with our data weights file with `pipeline.load_ckpt()` method:

In [13]:
# Load checkpoint using `load_ckpt` method (restoring weights for inference)
pipeline.load_ckpt(ckpt_path=ckpt_path)

INFO - 2022-04-04 17:06:01,353 - semantic_segmentation - Loading checkpoint ./randlanet_semantickitti_202201071330utc.pth


Now, lets query the first pointcloud from the `test` split.

In [16]:
test_data = dataset.get_split('test')
data = test_data.get_data(0)

INFO - 2022-04-04 17:13:43,378 - semantickitti - Found 1 pointclouds for test


Let's display the content `data` contains:

In [23]:
print(data)

{'point': array([[ 5.2305943e+01,  2.2989707e-02,  1.9779946e+00],
       [ 5.3259735e+01,  1.0695236e-01,  2.0099745e+00],
       [ 5.3284321e+01,  2.7487758e-01,  2.0109341e+00],
       ...,
       [ 3.8249431e+00, -1.4261885e+00, -1.7655631e+00],
       [ 3.8495324e+00, -1.4222100e+00, -1.7755738e+00],
       [ 3.8631279e+00, -1.4142324e+00, -1.7805853e+00]], dtype=float32), 'feat': None, 'label': array([0, 0, 0, ..., 9, 9, 9], dtype=int32)}


For inference on custom data, you can convert your pointcloud into this format:

**Dictionary with keys {'point', 'feat', 'label'}**

If you already have the *ground truth labels*, you can add them to data to get accuracy and IoU (Intersection over Union). Otherwise, pass labels as `None`.

And now - the main topic of our tutorial - running an inference on the test data. You can call the `run_inference()` method with your data, it will print accuracy per class and Intersection over Union (IoU) metrics. The last entry in the list is *mean accuracy* and *mean IoU*:

In [24]:
# Running inference on test data
results = pipeline.run_inference(data)
# prints per class accuracy and IoU (Intersection over Union). Last entry is mean accuracy and mean IoU.
# We get several `nan` outputs for missing classes in the input data.

test 0/1: 100%|████████████████████████▉| 83258/83500 [00:12<00:00, 6721.37it/s]INFO - 2022-04-04 17:28:38,137 - semantic_segmentation - Accuracy : [0.972646455436517, nan, nan, nan, nan, nan, nan, 0.0, 0.9576552007272514, 0.9159488384233881, 0.4735023041474654, nan, 0.9099061522419186, 0.9974093264248705, 0.8342307124944682, 0.8268921095008052, 0.9240506329113924, 0.9487179487179487, 0.67, 0.7859133067521687]
INFO - 2022-04-04 17:28:38,140 - semantic_segmentation - IoU : [0.8774419082870656, nan, 0.0, nan, nan, nan, nan, 0.0, 0.900385993934381, 0.21923028864175934, 0.4655992749792312, 0.0, 0.8965375526559128, 0.08926501275214468, 0.8220736272122687, 0.7171787709497207, 0.6831550802139037, 0.8712715855572999, 0.5826086956521739, 0.5089105564882759]
test 0/1: 100%|█████████████████████████| 83500/83500 [00:30<00:00, 6721.37it/s]

The `results` object will return dictionary of predicted labels and predicted probabilities per point:

In [25]:
# Dictionary of predicted labels and predicted probabilities per class
results

{'predict_labels': array([12, 12, 12, ...,  8,  8,  8], dtype=int16),
 'predict_scores': array([[1.061e-05, 6.437e-06, 2.980e-07, ..., 5.573e-05, 1.382e-03,
         7.524e-04],
        [6.795e-06, 7.331e-06, 2.980e-07, ..., 7.313e-05, 2.205e-03,
         8.492e-04],
        [6.199e-06, 6.318e-06, 2.980e-07, ..., 7.361e-05, 3.147e-03,
         1.205e-03],
        ...,
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00]], dtype=float16)}